In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the pixel values between 0 and 1
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert the labels to one-hot encoding
y_train = keras.utils.to_categorical(y_train, 100)
y_test = keras.utils.to_categorical(y_test, 100)


In [1]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tqdm import tqdm

# Define the path to the directory
directory_path = r"E:\imbcifar\train"

# Initialize empty lists for images and labels
X_train = []
Y_train = []
class_label = 0

# Loop through subdirectories (classes)
for class_folder in tqdm(sorted(os.listdir(directory_path))):
    class_path = os.path.join(directory_path, class_folder)
    
    for image_file in os.listdir(class_path):
        if image_file.endswith('.png'):
            image_path = os.path.join(class_path, image_file)
            
            # Load image, convert to RGB and resize
            img = Image.open(image_path).convert('RGB')
            img = img.resize((32, 32))
            img_array = np.array(img)
            
            # Append image and label to lists
            X_train.append(img_array)
            Y_train.append(class_label)
            
    class_label = class_label + 1

# Convert lists to numpy arrays
X_train = np.array(X_train)
Y_train = np.array(Y_train)

print(f'X_train shape: {X_train.shape}')
print(f'Y_train shape: {Y_train.shape}')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:09<00:00,  1.11it/s]

X_train shape: (29009, 32, 32, 3)
Y_train shape: (29009,)


In [1]:
from PIL import Image
import numpy as np
import os

def read_and_process_images(base_dir):
    images = []
    # Iterate through all subdirectories
    for subdir, dirs, files in os.walk(base_dir):
        for file in files:
            # Construct the full file path
            filepath = os.path.join(subdir, file)
            try:
                # Open the image
                with Image.open(filepath) as img:
                    # Convert to RGB
                    img = img.convert('RGB')
                    # Resize to 32x32
                    img = img.resize((32, 32), Image.ANTIALIAS)
                    # Convert to numpy array and scale
                    img_array = np.asarray(img, dtype=np.float32) / 255
                    images.append(img_array)
            except IOError:
                # Handle the case where the file could not be opened as an image
                print(f"Cannot open {file} as an image.")
                
    return images

base_dir = r"D:\datasets\Underwater_Image\WHOI\archive\dataset_pm\training"
X_train = read_and_process_images(base_dir)

C:\Users\shaif\AppData\Local\Temp\ipykernel_16216\1555111213.py:18: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize((32, 32), Image.ANTIALIAS)


Cannot open desktop.ini as an image.
Cannot open desktop.ini as an image.
Cannot open desktop.ini as an image.
Cannot open desktop.ini as an image.
Cannot open desktop.ini as an image.
Cannot open desktop.ini as an image.
Cannot open desktop.ini as an image.
Cannot open desktop.ini as an image.
Cannot open desktop.ini as an image.
Cannot open desktop.ini as an image.
Cannot open desktop.ini as an image.
Cannot open desktop.ini as an image.
Cannot open desktop.ini as an image.
Cannot open desktop.ini as an image.
Cannot open desktop.ini as an image.
Cannot open desktop.ini as an image.
Cannot open desktop.ini as an image.
Cannot open desktop.ini as an image.
Cannot open desktop.ini as an image.


In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Define the paths to your dataset folders
train_dataset_dir = r"D:\datasets\Underwater_Image\LIMUC (Labeled Images for Ulcerative Colitis)\train_and_validation_sets\train_and_validation_sets"
test_dataset_dir = r"D:\datasets\Underwater_Image\LIMUC (Labeled Images for Ulcerative Colitis)\test_set\test_set"

# Initialize empty lists for X_train, Y_train, X_test, and Y_test
X_train = []
Y_train = []
X_test = []
Y_test = []

# Initialize an empty list to store categorical labels
categorical_labels = []

# Define a function to read and preprocess images
def process_images(folder_path, label, is_train_set=True):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path) and filename.endswith(".bmp"):  # Check if it's a file and ends with .bmp
            # Open and resize the image to (32, 32, 3)
            img = Image.open(file_path)
            img = img.resize((32, 32))
            img = img.convert("RGB")
            
            # Convert image data to a NumPy array
            img_array = np.array(img).astype('float32')  # Convert to float
            
            # Normalize the image data (optional)
            img_array /= 255.0  # Normalize pixel values to [0, 1]
            
            # Append the image data to the appropriate list
            if is_train_set:
                X_train.append(img_array)
                Y_train.append(label)  # Append the numerical label
            else:
                X_test.append(img_array)
                Y_test.append(label)  # Append the numerical label
            
            # Append the label for categorical encoding
            categorical_labels.append(label)  # Append the numerical label

# List the folders inside the training dataset directory
train_folders = os.listdir(train_dataset_dir)

# Create a label encoder for categorical labels
label_encoder = LabelEncoder()

# Loop through the training folders and process images
for label, folder_name in enumerate(train_folders):
    folder_path = os.path.join(train_dataset_dir, folder_name)
    if os.path.isdir(folder_path):  # Check if it's a directory
        process_images(folder_path, label)

# List the folders inside the test dataset directory
test_folders = os.listdir(test_dataset_dir)

# Loop through the test folders and process images
for label, folder_name in enumerate(test_folders):
    folder_path = os.path.join(test_dataset_dir, folder_name)
    if os.path.isdir(folder_path):  # Check if it's a directory
        process_images(folder_path, label, is_train_set=False)
        
        
        
x_train = np.concatenate((X_train, X_test), axis=0)
# Check the shape of X_train, Y_train_categorical, X_test, and Y_test_categorical
print("Shape of X_train:", np.shape(x_train))

Shape of X_train: (11276, 32, 32, 3)


In [2]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
import glob

target_size = (32, 32)  # Change the values as per your requirement
# Load the pre-trained ResNet50 model with modified input shape
model = ResNet50(weights='imagenet', include_top=False, pooling='avg', input_shape=(target_size[0], target_size[1], 3))

In [7]:
import pandas as pd
# Define the file path
file_path = r"F:\feature.csv"
# Read the tab-separated CSV file into a DataFrame
df = pd.read_csv(file_path, delimiter='\t')
columns_to_drop = [df.columns[0], df.columns[-1]]
data = df.drop(columns_to_drop, axis=1)
# Display the head of the DataFrame
data.head(1)

,Feature 0,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5,Feature 6,Feature 7,Feature 8,Feature 9,...,Feature 2038,Feature 2039,Feature 2040,Feature 2041,Feature 2042,Feature 2043,Feature 2044,Feature 2045,Feature 2046,Feature 2047
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.009121,0.0,0.0,0.0,0.0,3.285227,0.0,0.0


In [4]:
np.shape(X_train)

(4357, 32, 32, 3)

In [4]:
from PIL import Image
import numpy as np
ft = model.predict(np.array(X_train).astype("float32"))

In [5]:
from sklearn.cluster import MiniBatchKMeans

n_clusters = 15
batch_size = 100
max_iter = 100

kmeans = MiniBatchKMeans(n_clusters=n_clusters, batch_size=batch_size, max_iter=max_iter)
kmeans.fit(ft)
# Retrieve the cluster centers
ct = kmeans.cluster_centers_.tolist()

C:\Users\shaif\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\shaif\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1902: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5120 or by setting the environment variable OMP_NUM_THREADS=1
  warnings.warn(


In [6]:
from tqdm import tqdm

combo_list = []  # Initialize combo_list
tot_dist = []
# Iterate over ft
for i in tqdm(range(len(ct))):
    distances = []

    # Calculate distances for each row in data
    for index, row in data.iterrows():
        row_array = row.to_numpy()  # Convert row to numpy array
        distance = np.linalg.norm(ct[i] - row_array)  # Calculate Euclidean distance
        distances.append(distance)

    tot_dist.append(distances)
print(np.shape(tot_dist))
#new_shape = (1000, 30)
#re_dist = np.transpose(tot_dist, (1, 0))
re_dist = tot_dist
print(np.shape(re_dist))

100%|███████████████████████████████████████████████████████████| 15/15 [00:33<00:00,  2.25s/it]

(15, 10450)
(15, 10450)


In [7]:
import pickle

# Saving the list to a file in the D: drive
file_path = r"F:\dist_list.pickle"  # r prefix is used for raw string to avoid escape characters

with open(file_path, 'wb') as file:
    pickle.dump(re_dist, file)

print("Combo list saved successfully.")

Combo list saved successfully.


In [7]:
len(index_list)

400

In [6]:
import pickle

# Loading the list from the pickle file
file_path = r"F:\index_list.pickle"  # Update the file path if necessary

with open(file_path, 'rb') as file:
    index_list = pickle.load(file)

print("Combo list loaded successfully.")
print(index_list)


Combo list loaded successfully.
[7, 16, 18, 19, 21, 22, 23, 51, 87, 88, 89, 93, 95, 96, 105, 112, 117, 121, 122, 125, 130, 132, 144, 145, 151, 152, 153, 154, 155, 156, 157, 158, 161, 162, 164, 165, 166, 167, 169, 173, 174, 175, 176, 179, 180, 181, 183, 188, 190, 194, 195, 196, 197, 198, 199, 200, 201, 204, 205, 206, 208, 212, 214, 215, 216, 217, 218, 219, 221, 222, 223, 224, 226, 227, 228, 229, 230, 231, 232, 233, 234, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 256, 257, 258, 261, 262, 264, 265, 266, 267, 268, 284, 295, 297, 302, 306, 321, 324, 332, 338, 355, 359, 361, 364, 366, 367, 368, 369, 370, 374, 375, 377, 378, 379, 380, 381, 382, 383, 384, 387, 388, 389, 391, 392, 399, 400, 401, 402, 409, 413, 414, 417, 420, 422, 423, 424, 426, 429, 432, 433, 439, 440, 441, 442, 445, 446, 447, 448, 450, 451, 452, 456, 457, 459, 461, 464, 465, 466, 469, 473, 475, 477, 479, 486, 487, 488, 494, 496, 499, 501, 502, 504, 505, 506, 507, 512, 513, 51

In [5]:
df.columns

Index(['Directory', 'Feature 0', 'Feature 1', 'Feature 2', 'Feature 3',
       'Feature 4', 'Feature 5', 'Feature 6', 'Feature 7', 'Feature 8',
       ...
       'Feature 2039', 'Feature 2040', 'Feature 2041', 'Feature 2042',
       'Feature 2043', 'Feature 2044', 'Feature 2045', 'Feature 2046',
       'Feature 2047', 'Unnamed: 2049'],
      dtype='object', length=2050)

In [10]:
selected_data = df.loc[index_list, ['Name']].to_numpy().flatten()

In [11]:
import pickle

# Saving the list to a file in the D: drive
file_path = r"F:\combo_list.pickle"  # r prefix is used for raw string to avoid escape characters

with open(file_path, 'wb') as file:
    pickle.dump(selected_data, file)

print("Combo list saved successfully.")


Combo list saved successfully.


In [12]:
import pickle

# Loading the list from the pickle file
file_path = r"F:\combo_list.pickle"  # Update the file path if necessary

with open(file_path, 'rb') as file:
    loaded_combo_list = pickle.load(file)

print("Combo list loaded successfully.")
print(loaded_combo_list)


Combo list loaded successfully.
['D:/data/imagenet\\n01514668\\' 'D:/data/imagenet\\n01560419\\'
 'D:/data/imagenet\\n01582220\\' 'D:/data/imagenet\\n01592084\\'
 'D:/data/imagenet\\n01608432\\' 'D:/data/imagenet\\n01614925\\'
 'D:/data/imagenet\\n01616318\\' 'D:/data/imagenet\\n01704323\\'
 'D:/data/imagenet\\n01817953\\' 'D:/data/imagenet\\n01818515\\'
 'D:/data/imagenet\\n01819313\\' 'D:/data/imagenet\\n01829413\\'
 'D:/data/imagenet\\n01843065\\' 'D:/data/imagenet\\n01843383\\'
 'D:/data/imagenet\\n01882714\\' 'D:/data/imagenet\\n01943899\\'
 'D:/data/imagenet\\n01968897\\' 'D:/data/imagenet\\n01981276\\'
 'D:/data/imagenet\\n01983481\\' 'D:/data/imagenet\\n01986214\\'
 'D:/data/imagenet\\n02007558\\' 'D:/data/imagenet\\n02009912\\'
 'D:/data/imagenet\\n02051845\\' 'D:/data/imagenet\\n02056570\\'
 'D:/data/imagenet\\n02085620\\' 'D:/data/imagenet\\n02085782\\'
 'D:/data/imagenet\\n02085936\\' 'D:/data/imagenet\\n02086079\\'
 'D:/data/imagenet\\n02086240\\' 'D:/data/imagenet\\n02086

In [5]:
import os

file_path = r"F:\D-Video\Python"

# Check if the file exists
if os.path.exists(file_path):
    print(f"File exists: {file_path}")
else:
    print(f"File does not exist: {file_path}")

# Check if the file is readable
if os.access(file_path, os.R_OK):
    print(f"File is readable: {file_path}")
else:
    print(f"File is not readable: {file_path}")


File exists: F:\D-Video\Python
File is readable: F:\D-Video\Python
